## Multi-label Legal Text Classification for CIA

### *OCR for scanned PDFs*

In [73]:
!pip install tesseract
!pip install tesseract-ocr
!pip install opencv-python
!pip install pytesseract
!pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 10.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for tesseract: filename=tesseract-0.1.3-py3-none-any.whl size=45562552 sha256=eeca1aa32fafcab9d37345aabc507a730fa4ceb6717636f360ab0fc9c18dd1f8
  Stored in directory: /Users/janinedevera/Library/Caches/pip/wheels/d6/d9/8d/8f25140c367f80dac44190e77ea4934ddc1ad9b4382a2f79e2
Successfully built tesseract

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      /Users/janinedevera/opt/miniconda3/envs/watermelon/lib/python3.9/site-packages/setuptools/dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' in

In [75]:
import cv2
import pytesseract
import re
from pdf2image import convert_from_bytes, convert_from_path

In [76]:
from glob import glob
import os
import numpy as np
import pandas as pd

In [ ]:
os.chdir("../..")

In [77]:
ocr_dir = "data/legislation/For OCR/"
ocr_list = glob(os.path.join(ocr_dir, "*.pdf"))
len(ocr_list)

29

Help functions

In [78]:
# average confidence value
def get_conf(page_gray):
    df = pytesseract.image_to_data(page_gray,output_type='data.frame')
    df.drop(df[df.conf==-1].index.values,inplace=True)
    df.reset_index()
    return df.conf.mean()

# deskewing image
def deskew(image):
    gray = cv2.bitwise_not(image)
    temp_arr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(temp_arr > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

Main OCR

In [ ]:
OCR_dic={} 
for file in ocr_list:
    # convert pdf into image
    pdf_file = convert_from_path(file)
    # create a df to save each pdf's text
    pages_df = pd.DataFrame(columns=['conf','text'])
    for (i,page) in enumerate(pdf_file) :
        #try:
            # transfer image of pdf_file into array
            page_arr = np.asarray(page)
            # transfer into grayscale
            page_arr_gray = cv2.cvtColor(page_arr,cv2.COLOR_BGR2GRAY)
            # deskew the page
            page_deskew = deskew(page_arr_gray)
            # cal confidence value
            page_conf = get_conf(page_deskew)
            # extract string 
            pages_df = pages_df.append({'conf': page_conf,'text': pytesseract.image_to_string(page_deskew)}, ignore_index=True)
        #except:
            # if can't extract then give some notes into df
            #pages_df = pages_df.append({'conf': -1,'text': 'N/A'}, ignore_index=True)
            #continue
    # save df into a dict with filename as key        
    OCR_dic[file]=pages_df
    print('{} is done'.format(file))

In [97]:
OCR_dic[ocr_list[28]]['text'][1]

'171.5\n\n11.6\n\n3.0 Effectivity\n\n@Ppropriate human resource complement for their\nPrograms/activities/projects\n\nOve\n(GCG) ang to existing budgeting and 4ccounting Tules and regulations.\n\nwith the agency. The Payment of Services shalf be Charged against\nthe Maintenance and Other Operating Expenses in the approved\nagency budget.\n\nPayment of Services of Persons or entities engaged through Republic\nAct No. 91 84 shall be Subject to the Provisions of the said law and its\nimplementing tules and regulations.”\n\nALICIA dela ROSA-BALA MICH fi\nChairperson Chairperson\nCivil Service Commission Commission on Audit\n\n'